Access and plot the fields from the World Ocean Atlas

__Notes:__ it might be better to download all the datasets because the connection to the THREDDS server is rather slow.

In [54]:
using NCDatasets
using PyPlot
const plt = PyPlot

PyPlot

 ## Settings

In [1]:
resolution = 0.25
month = 1
variable = "temperature"
baseurl = "https://www.ncei.noaa.gov/thredds-ocean/catalog/ncei/woa/$(variable)"

"https://www.ncei.noaa.gov/thredds-ocean/catalog/ncei/woa/temperature"

## Create data URL

In [9]:
monthnum = lpad(month, 2, "0")

dataurlref = "https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/temperature/6574/0.25/woa18_6574_t13_04.nc"
dataurl = "$(baseurl)/A5B7/0.25/catalog.html?dataset=ncei/woa/$(variable)/A5B7/$(resolution)/woa18_A5B7_t$(monthnum)_04.nc"

"https://www.ncei.noaa.gov/thredds-ocean/catalog/ncei/woa/temperature/A5B7/0.25/catalog.html?dataset=ncei/woa/temperature/A5B7/0.25/woa18_A5B7_t01_04.nc"

In [10]:
try 
    NCDatasets.Dataset(dataurlref) do nc
        print(nc)
    end
catch NetCDF error
    @warn("NetCDF: Malformed or unexpected Constraint")
end

NCDataset: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/temperature/6574/0.25/woa18_6574_t13_04.nc
Group: /

Dimensions
   depth = 102
   lat = 720
   lon = 1440
   nbounds = 2
   time = 1

Variables
  crs  
    Attributes:
     grid_mapping_name    = latitude_longitude
     epsg_code            = EPSG:4326
     longitude_of_prime_meridian = 0.0
     semi_major_axis      = 6.378137e6
     inverse_flattening   = 298.25723

  lat   (720)
    Datatype:    Float32
    Dimensions:  lat
    Attributes:
     standard_name        = latitude
     long_name            = latitude
     units                = degrees_north
     axis                 = Y
     bounds               = lat_bnds
     _ChunkSizes          = 720

  lat_bnds   (2 × 720)
    Datatype:    Float32
    Dimensions:  nbounds × lat
    Attributes:
     comment              = latitude bounds
     _ChunkSizes          = Int32[720, 2]

  lon   (1440)
    Datatype:    Float32
    Dimensions:  lon
    Attributes:
     standard

  creator_email        = NCEI.info@noaa.gov
  creator_url          = http://www.ncei.noaa.gov
  creator_type         = group
  creator_institution  = National Centers for Environmental Information
  project              = World Ocean Atlas Project
  processing_level     = processed
  keywords             = Oceans< Ocean Temperature > Water Temperature
  keywords_vocabulary  = ISO 19115
  standard_name_vocabulary = CF Standard Name Table v49
  contributor_name     = Ocean Climate Laboratory
  contributor_role     = Calculation of climatologies
  cdm_data_type        = Grid
  publisher_name       = National Centers for Environmental Information (NCEI)
  publisher_institution = National Centers for Environmental Information
  publisher_type       = institution
  publisher_url        = http://www.ncei.noaa.gov/
  publisher_email      = NCEI.info@noaa.gov
  nodc_template_version = NODC_NetCDF_Grid_Template_v2.0
  license              = These data are openly available to the public. Please a

In [52]:
function read_data_WOA(dataurl::String, domain=[-180., 180., -90., 90.], depthlevel=5.)
    NCDatasets.Dataset(dataurlref) do nc
        lon = varbyattrib(nc, standard_name = "longitude")[1][:]
        lat = varbyattrib(nc, standard_name = "longitude")[1][:]
        depth = varbyattrib(nc, standard_name = "depth")[1][:]
        #time = varbyattrib(nc, standard_name = "time")[1][:]
        

        # Subsetting
        goodlon = findall((lon .<= domain[2]) .& (lon .>= domain[1]))
        goodlat = findall((lat .<= domain[4]) .& (lat .>= domain[3]))
        gooddepth = findall(depth .== depthlevel)
        
        @info(goodlon);
        @info("Good lat: $(goodlat)");
        @info(gooddepth);
        
        temperature = varbyattrib(nc, standard_name = "sea_water_temperature")[1][goodlat, goodlon, 1, 1]
        
        return temperature #lon[goodlon], lat[goodlat], depth[gooddepth], temperature
    end
end

read_data_WOA (generic function with 3 methods)

In [56]:
domain = [-30, -10., 25., 35.]
T = read_data_WOA(dataurlref, domain, 5.);

┌ Info: [601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680]
└ @ Main In[52]:14
┌ Info: Good lat: [821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860]
└ @ Main In[52]:15
┌ Info: [2]
└ @ Main In[52]:16
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <!DOCTYPE^ HTML PUBLIC "-//IETF//DTD HTML 2.0//EN"><html><head><title>502 Proxy Error</title></head><body><h1>Proxy Error</h1><p>The proxy server received an invalidresponse from an upstream server.<br />The 

LoadError: NetCDF error: [31mNetCDF: DAP server error[39m (NetCDF error code: -70)

In [55]:
plt.pcolormesh(T)
plt.show()

LoadError: PyError ($(Expr(:escape, :(ccall(#= /home/ctroupin/.julia/packages/PyCall/3fwVL/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('not enough values to unpack (expected 2, got 1)')
  File "/home/ctroupin/Software/PythonEnvs/EddyMitchell/lib/python3.8/site-packages/matplotlib/pyplot.py", line 2785, in pcolormesh
    __ret = gca().pcolormesh(
  File "/home/ctroupin/Software/PythonEnvs/EddyMitchell/lib/python3.8/site-packages/matplotlib/__init__.py", line 1438, in inner
    return func(ax, *map(sanitize_sequence, args), **kwargs)
  File "/home/ctroupin/Software/PythonEnvs/EddyMitchell/lib/python3.8/site-packages/matplotlib/axes/_axes.py", line 6085, in pcolormesh
    X, Y, C, shading = self._pcolorargs('pcolormesh', *args,
  File "/home/ctroupin/Software/PythonEnvs/EddyMitchell/lib/python3.8/site-packages/matplotlib/axes/_axes.py", line 5557, in _pcolorargs
    nrows, ncols = C.shape
